In [135]:
from rdflib import Graph
import pandas as pd
import ssl

#ctx = ssl.create_default_context()
#ctx.check_hostname = False
#ctx.verify_mode = ssl.CERT_NONE

ssl._create_default_https_context = ssl._create_unverified_context

import urllib.parse
def extract_last_part(url):
    last_part = url.split("/")[-1]
    return urllib.parse.unquote(last_part) # sonderzeichen anzeigen lassen

# Grundlageninformationen zu Gemeinden und Kantonen
Datenquelle: das Geoportal des Bundes (https://geo.ld.admin.ch/) und wikidata (https://query.wikidata.org/)

### Abfrage: Gemeinde, Gemeindeeinwohnerzahl, Kanton, Kantonseinwohnerzahl

In [171]:
GemeindeKantone_QUERY = '''
PREFIX dv: <http://rdf.data-vocabulary.org/#>
PREFIX schema: <http://schema.org/>
PREFIX gn: <http://www.geonames.org/ontology#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>

SELECT  ?Date ?Gemeinde_Entity ?Gemeinde_Name ?Gemeinde_Population ?Kanton_Name ?Kanton_Population
WHERE {
        SERVICE <https://geo.ld.admin.ch/query>
            {   ?Gemeinde gn:featureCode gn:A.ADM3 .
                ?Gemeinde schema:name ?Gemeinde_Name .
                ?Gemeinde gn:population ?Gemeinde_Population .
                ?Gemeinde dc:issued ?Date .
  				?Gemeinde dc:isVersionOf ?Gemeinde_Version .
  				?Gemeinde_Version rdfs:seeAlso ?Gemeinde_Entity .

                ?Gemeinde gn:parentADM1 ?Canton .
                ?Canton schema:name ?Kanton_Name .
                ?Canton gn:population ?Kanton_Population .

                FILTER (?Date > "2009-01-01"^^xsd:date)
                }
        }
        '''

# Graph erstellen und SPARQL-Abfrage durchführen
g = Graph()
results = g.query(GemeindeKantone_QUERY)

result_list = []
for row in results:
    result_list.append(row)

# Liste von Tupeln in ein Pandas DataFrame umwandeln
df_g = pd.DataFrame(result_list, columns=['Date', 'Gemeinde_Entity', 'Gemeinde_Name', 'Gemeinde_Population', 'Kanton_Name', 'Kanton_Population'])

# DataFrame anzeigen
# df_g


In [172]:
df_g

,Date,Gemeinde_Entity,Gemeinde_Name,Gemeinde_Population,Kanton_Name,Kanton_Population
0,2022-01-01,http://www.wikidata.org/entity/Q64945,Aeugst am Albis,2014,Zürich,1553423
1,2022-01-01,http://www.wikidata.org/entity/Q66422,Ferpicloz,267,Fribourg,325547
2,2022-01-01,http://www.wikidata.org/entity/Q67720,Randa,425,Valais,348503
3,2022-01-01,http://www.wikidata.org/entity/Q52777,Chéserex,1249,Vaud,814762
4,2022-01-01,http://www.wikidata.org/entity/Q65620,Tuggen,3317,Schwyz,162157
...,...,...,...,...,...,...
25362,2013-01-01,http://www.wikidata.org/entity/Q66757,Flurlingen,1429,Zürich,1392396
25363,2013-01-01,http://www.wikidata.org/entity/Q68210,Adliswil,17518,Zürich,1392396
25364,2013-01-01,http://www.wikidata.org/entity/Q64380,Gersau,2108,Schwyz,147904
25365,2013-01-01,http://www.wikidata.org/entity/Q66892,Hütten,911,Zürich,1392396


In [173]:
def extract_year(date_str):
    return date_str[:4]

# Nur das Jahr extrahieren und in eine neue Spalte einfügen
df_g['Date'] = df_g['Date'].apply(extract_year)

In [174]:
df_g

,Date,Gemeinde_Entity,Gemeinde_Name,Gemeinde_Population,Kanton_Name,Kanton_Population
0,2022,http://www.wikidata.org/entity/Q64945,Aeugst am Albis,2014,Zürich,1553423
1,2022,http://www.wikidata.org/entity/Q66422,Ferpicloz,267,Fribourg,325547
2,2022,http://www.wikidata.org/entity/Q67720,Randa,425,Valais,348503
3,2022,http://www.wikidata.org/entity/Q52777,Chéserex,1249,Vaud,814762
4,2022,http://www.wikidata.org/entity/Q65620,Tuggen,3317,Schwyz,162157
...,...,...,...,...,...,...
25362,2013,http://www.wikidata.org/entity/Q66757,Flurlingen,1429,Zürich,1392396
25363,2013,http://www.wikidata.org/entity/Q68210,Adliswil,17518,Zürich,1392396
25364,2013,http://www.wikidata.org/entity/Q64380,Gersau,2108,Schwyz,147904
25365,2013,http://www.wikidata.org/entity/Q66892,Hütten,911,Zürich,1392396


### Abfrage: PLZ

In [175]:
PLZ_QUERY = '''
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>

SELECT  ?Gemeinde_Entity ?PLZ
WHERE {
        SERVICE <https://query.wikidata.org/sparql>
            {   ?Gemeinde_Entity    wdt:P31     wd:Q70208 ;  # Filter für Ortschaften
                                    wdt:P17     wd:Q39 ;     # Filter für die Schweiz als Herkunftsland
                                    wdt:P281    ?PLZ .
            }
        }
        '''
# Graph erstellen und SPARQL-Abfrage durchführen
g = Graph()
results_plz = g.query(PLZ_QUERY)

result_plz_list = []
for row in results_plz:
    result_plz_list.append(row)

# Liste von Tupeln in ein Pandas DataFrame umwandeln
df_plz_g = pd.DataFrame(result_plz_list, columns=['Gemeinde_Entity', 'PLZ'])

# DataFrame anzeigen
# df_plz_g

# -> einige PLZ sind in einem Intervall angegeben

In [176]:
# PLZ intervall auflösen und in neue Zeilen speicher
def split_and_explode(row):
    plz_range = row['PLZ'].split('–')
    if len(plz_range) == 2:
        start, end = map(int, plz_range)
        return [(row['Gemeinde_Entity'], str(i)) for i in range(start, end + 1)]
    else:
        return [(row['Gemeinde_Entity'], row['PLZ'])]

new_rows = []
for _, row in df_plz_g.iterrows():
    new_rows.extend(split_and_explode(row))

# Neues DataFrame erstellen
df_plz_g_all = pd.DataFrame(new_rows, columns=['Gemeinde_Entity', 'PLZ'])
#df_plz_g_all

### Abfrage: Kanton und deren Abkürzungen

In [177]:
KurznamenKanton_QUERY = '''
PREFIX schema: <http://schema.org/>

select distinct ?Kanton_Kurzname ?Kanton_Name
where {
        SERVICE <https://geo.ld.admin.ch/query>
            {?Canton schema:containedInPlace ?Land.
            ?Canton schema:name ?Kanton_Name .
            ?Canton schema:alternateName ?Kanton_Kurzname

        FILTER (lang(?Kanton_Name) ='de' || lang(?Kanton_Name) ='fr' || lang(?Kanton_Name) ='it')
}}
'''
g2 = Graph()
results_g2 = g2.query(KurznamenKanton_QUERY)

result_g2_list = []
for row in results_g2:
    result_g2_list.append(row)

# Liste von Tupeln in ein Pandas DataFrame umwandeln
df_g2 = pd.DataFrame(result_g2_list, columns=['Kanton_Kurzname', 'Kanton_Name',])

# DataFrame anzeigen
# df_g2

### Tabellen Merge

In [178]:
# whitespace entfernen in den Kanton_Namen
df_g['Kanton_Name'] = df_g['Kanton_Name'].str.replace('\s', '', regex=True)
df_g2['Kanton_Name'] = df_g2['Kanton_Name'].str.replace('\s', '', regex=True)

In [179]:
# Zusammenführen der Listen
# Basisliste ist df_g

df_grundlagen_z = pd.merge(df_g, df_g2, on='Kanton_Name', how='left')
df_grundlagen_z

,Date,Gemeinde_Entity,Gemeinde_Name,Gemeinde_Population,Kanton_Name,Kanton_Population,Kanton_Kurzname
0,2022,http://www.wikidata.org/entity/Q64945,Aeugst am Albis,2014,Zürich,1553423,ZH
1,2022,http://www.wikidata.org/entity/Q66422,Ferpicloz,267,Fribourg,325547,FR
2,2022,http://www.wikidata.org/entity/Q67720,Randa,425,Valais,348503,VS
3,2022,http://www.wikidata.org/entity/Q52777,Chéserex,1249,Vaud,814762,VD
4,2022,http://www.wikidata.org/entity/Q52777,Chéserex,1249,Vaud,814762,VD
...,...,...,...,...,...,...,...
30163,2013,http://www.wikidata.org/entity/Q66757,Flurlingen,1429,Zürich,1392396,ZH
30164,2013,http://www.wikidata.org/entity/Q68210,Adliswil,17518,Zürich,1392396,ZH
30165,2013,http://www.wikidata.org/entity/Q64380,Gersau,2108,Schwyz,147904,SZ
30166,2013,http://www.wikidata.org/entity/Q66892,Hütten,911,Zürich,1392396,ZH


In [180]:
df_grundlagen = pd.merge(df_grundlagen_z, df_plz_g_all, on='Gemeinde_Entity', how='left')
df_grundlagen

,Date,Gemeinde_Entity,Gemeinde_Name,Gemeinde_Population,Kanton_Name,Kanton_Population,Kanton_Kurzname,PLZ
0,2022,http://www.wikidata.org/entity/Q64945,Aeugst am Albis,2014,Zürich,1553423,ZH,NaN
1,2022,http://www.wikidata.org/entity/Q66422,Ferpicloz,267,Fribourg,325547,FR,1724
2,2022,http://www.wikidata.org/entity/Q67720,Randa,425,Valais,348503,VS,3928
3,2022,http://www.wikidata.org/entity/Q52777,Chéserex,1249,Vaud,814762,VD,1275
4,2022,http://www.wikidata.org/entity/Q52777,Chéserex,1249,Vaud,814762,VD,1275
...,...,...,...,...,...,...,...,...
36375,2013,http://www.wikidata.org/entity/Q66757,Flurlingen,1429,Zürich,1392396,ZH,NaN
36376,2013,http://www.wikidata.org/entity/Q68210,Adliswil,17518,Zürich,1392396,ZH,8134
36377,2013,http://www.wikidata.org/entity/Q64380,Gersau,2108,Schwyz,147904,SZ,6442
36378,2013,http://www.wikidata.org/entity/Q66892,Hütten,911,Zürich,1392396,ZH,NaN


In [181]:
# in der Kanton_Kurznamenliste sind Basel-Stadt und Basel-Land nicht enthalten, weswegen diese hier noch ergänzt werden
df_grundlagen['Kanton_Kurzname'].mask(df_grundlagen['Kanton_Name'] == 'Basel-Stadt', 'BS', inplace=True)
df_grundlagen['Kanton_Kurzname'].mask(df_grundlagen['Kanton_Name'] == 'Basel-Landschaft', 'BL', inplace=True)

In [182]:
# Speichern Sie das DataFrame als CSV-Datei
df_grundlagen.to_csv('grundlagen_gemeinde_kantone.csv', index=False)

# Informationen zum Gebäudeprogramm - in Regionen unterteilt
Datenquelle: das Geoportal des Bundes (https://lindas.admin.ch/query)



In [207]:
kanton_dict_lindas ={
    '^https://ld.admin.ch/canton/1$' : 'ZH',
    '^https://ld.admin.ch/canton/2$' : 'BE',
    '^https://ld.admin.ch/canton/3$' : 'LU',
    '^https://ld.admin.ch/canton/4$' : 'UR',
    '^https://ld.admin.ch/canton/5$' : 'SZ',
    '^https://ld.admin.ch/canton/6$' : 'OW',
    '^https://ld.admin.ch/canton/7$' : 'NW',
    '^https://ld.admin.ch/canton/8$' : 'GL',
    '^https://ld.admin.ch/canton/9$' : 'ZG',
    '^https://ld.admin.ch/canton/10$' : 'FR',
    '^https://ld.admin.ch/canton/11$' : 'SO',
    '^https://ld.admin.ch/canton/12$' : 'BS',
    '^https://ld.admin.ch/canton/13$' : 'BL',
    '^https://ld.admin.ch/canton/14$' : 'SH',
    '^https://ld.admin.ch/canton/15$' : 'AR',
    '^https://ld.admin.ch/canton/16$' : 'AI',
    '^https://ld.admin.ch/canton/17$' : 'SG',
    '^https://ld.admin.ch/canton/18$' : 'GR',
    '^https://ld.admin.ch/canton/19$' : 'AG',
    '^https://ld.admin.ch/canton/20$' : 'TG',
    '^https://ld.admin.ch/canton/21$' : 'TI',
    '^https://ld.admin.ch/canton/22$' : 'VD',
    '^https://ld.admin.ch/canton/23$' : 'VS',
    '^https://ld.admin.ch/canton/24$' : 'NE',
    '^https://ld.admin.ch/canton/25$' : 'GE',
    '^https://ld.admin.ch/canton/26$' : 'JU',
    '^https://ld.admin.ch/country/CHE$' : 'CH'
}

### Abfrage: Anzahl Gesuche pro Jahr, Region und Art für Förderprogramm

In [208]:
Gutsprachen_QUERY = '''
PREFIX schema: <http://schema.org/>

select distinct ?Kanton_Kurzname ?Date ?Anzahl_Gesuche_mit_Auszahlung ?Massnahmenart
where {
        SERVICE <https://lindas.admin.ch/query>
            {?s schema:eligibleRegion ?Kanton_Kurzname.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_anzahl_gesuche/Jahr> ?Date.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_anzahl_gesuche/anzahl-gesuche-mit-auszahlungen> ?Anzahl_Gesuche_mit_Auszahlung.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_anzahl_gesuche/massnahmeart> ?Massnahmenart
}}
'''

# Graph erstellen und SPARQL-Abfrage durchführen
g3 = Graph()
results_g3 = g3.query(Gutsprachen_QUERY)

result_g3_list = []
for row in results_g3:
    result_g3_list.append(row)

# Liste in ein Pandas DataFrame umwandeln
df_g3 = pd.DataFrame(result_g3_list, columns=['Kanton_Kurzname', 'Date', 'Anzahl_Gesuche_mit_Auszahlung', 'Massnahmenart'])

# DataFrame anzeigen
df_g3

,Kanton_Kurzname,Date,Anzahl_Gesuche_mit_Auszahlung,Massnahmenart
0,CH,2010,0,Direkt
1,CH,2010,5514,Indirekt
2,CH,2011,0,Direkt
3,CH,2011,20547,Indirekt
4,CH,2012,0,Direkt
...,...,...,...,...
359,https://ld.admin.ch/canton/4,2022,147,Direkt
360,https://ld.admin.ch/canton/22,2022,2151,Direkt
361,https://ld.admin.ch/canton/23,2022,1277,Direkt
362,https://ld.admin.ch/canton/9,2022,104,Direkt


In [209]:
# nicht überall sind die Kurznamen hinterlegt, weshalb diese ersetzt werden
df_g3['Kanton_Kurzname'] = df_g3['Kanton_Kurzname'].replace(kanton_dict_lindas.keys() , kanton_dict_lindas.values() , regex=True)

In [210]:
# Trotz distinct hat es einige Doublikate, diese werden gelöscht
df_g3_no_duplicates = df_g3.drop_duplicates()
df_g3_no_duplicates

,Kanton_Kurzname,Date,Anzahl_Gesuche_mit_Auszahlung,Massnahmenart
0,CH,2010,0,Direkt
1,CH,2010,5514,Indirekt
2,CH,2011,0,Direkt
3,CH,2011,20547,Indirekt
4,CH,2012,0,Direkt
...,...,...,...,...
177,UR,2022,147,Direkt
178,VD,2022,2151,Direkt
179,VS,2022,1277,Direkt
180,ZG,2022,104,Direkt


In [211]:
# pro Kanton und Jahr aufsummieren
df_g3_grouped = df_g3_no_duplicates.groupby(['Kanton_Kurzname', 'Date'], as_index=False)['Anzahl_Gesuche_mit_Auszahlung'].sum()
df_g3_grouped

,Kanton_Kurzname,Date,Anzahl_Gesuche_mit_Auszahlung
0,AG,2017,267
1,AG,2018,400
2,AG,2019,786
3,AG,2020,745
4,AG,2021,1036
...,...,...,...
164,ZH,2018,965
165,ZH,2019,1400
166,ZH,2020,1513
167,ZH,2021,2680


In [212]:
df_g3_grouped.to_csv('gebaudeprogramm_AnzahlGesuche.csv', index=False)

### Abfrage: Summe Auszahlungen pro Jahr, Region und Bereich für Förderprogramm

In [213]:
Gebaeudeprogramm_QUERY = '''
PREFIX schema: <http://schema.org/>

select distinct ?Date ?Kanton_Kurzname ?VergütungCHF_Gebäudeprogramm ?Massnahmenbereich

where {
        SERVICE <https://lindas.admin.ch/query>
            {?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_auszahlungen/Jahr> ?Date.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_auszahlungen/region> ?Kanton_Kurzname.
            ?s <http://schema.org/amount> ?VergütungCHF_Gebäudeprogramm.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_auszahlungen/massnahmenbereich> ?Massnahmenbereich.
}}
'''

# Graph erstellen und SPARQL-Abfrage durchführen
g5 = Graph()
results_g5 = g5.query(Gebaeudeprogramm_QUERY)

result_g5_list = []
for row in results_g5:
    result_g5_list.append(row)

# Liste in ein Pandas DataFrame umwandeln
df_g5 = pd.DataFrame(result_g5_list, columns=['Date', 'Kanton_Kurzname', 'VergütungCHF_Gebäudeprogramm', 'Massnahmenbereich_lang'])
df_g5

,Date,Kanton_Kurzname,VergütungCHF_Gebäudeprogramm,Massnahmenbereich_lang
0,2010,https://ld.admin.ch/country/CHE,43522889.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
1,2010,https://ld.admin.ch/country/CHE,0.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
2,2010,https://ld.admin.ch/country/CHE,10977986.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
3,2010,https://ld.admin.ch/country/CHE,3501873.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
4,2010,https://ld.admin.ch/country/CHE,92235671.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
...,...,...,...,...
7197,2022,https://ld.admin.ch/canton/4,65390.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
7198,2022,https://ld.admin.ch/canton/22,1616377.3,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
7199,2022,https://ld.admin.ch/canton/23,27137.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
7200,2022,https://ld.admin.ch/canton/9,103693.85,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...


In [214]:
# nicht überall sind die Kurznamen hinterlegt, weshalb diese ersetzt werden
df_g5['Kanton_Kurzname'] = df_g5['Kanton_Kurzname'].replace(kanton_dict_lindas.keys() , kanton_dict_lindas.values() , regex=True)

In [215]:
df_g5['Massnahmenbereich'] = df_g5['Massnahmenbereich_lang'].apply(extract_last_part)
df_g5

,Date,Kanton_Kurzname,VergütungCHF_Gebäudeprogramm,Massnahmenbereich_lang,Massnahmenbereich
0,2010,CH,43522889.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Haustechnik
1,2010,CH,0.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte Massnahmen
2,2010,CH,10977986.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Neubau
3,2010,CH,3501873.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Systemsanierung
4,2010,CH,92235671.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Total
...,...,...,...,...,...
7197,2022,UR,65390.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen
7198,2022,VD,1616377.3,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen
7199,2022,VS,27137.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen
7200,2022,ZG,103693.85,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen


In [216]:
# Trotz distinct hat es einige Doublikate, diese werden gelöscht
df_g5_no_duplicates = df_g5.drop_duplicates()
df_g5_no_duplicates

,Date,Kanton_Kurzname,VergütungCHF_Gebäudeprogramm,Massnahmenbereich_lang,Massnahmenbereich
0,2010,CH,43522889.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Haustechnik
1,2010,CH,0.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte Massnahmen
2,2010,CH,10977986.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Neubau
3,2010,CH,3501873.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Systemsanierung
4,2010,CH,92235671.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Total
...,...,...,...,...,...
7197,2022,UR,65390.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen
7198,2022,VD,1616377.3,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen
7199,2022,VS,27137.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen
7200,2022,ZG,103693.85,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen


In [217]:
df_g5_no_duplicates.to_csv('gebaudeprogramm_SummeAuszahlungen.csv', index=False)

### Abfrage: Energiewirkung

In [218]:
Gebaeudeprogramm_Energiewirkung_QUERY = '''
PREFIX schema: <http://schema.org/>

select distinct  ?Date ?Kanton_Kurzname ?Energiewirkung_GWh_CH ?Massnahmenbereich

where {
        SERVICE <https://lindas.admin.ch/query>
            {?Massnahmenbereich <http://schema.org/amount> ?Energiewirkung_GWh_CH .
            ?Massnahmenbereich <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_energiewirkung/Jahr> ?Date .
            ?Massnahmenbereich <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_energiewirkung/region> ?Kanton_Kurzname.
}}
'''

# Graph erstellen und SPARQL-Abfrage durchführen
g6 = Graph()
results_g6 = g6.query(Gebaeudeprogramm_Energiewirkung_QUERY)

result_g6_list = []
for row in results_g6:
    result_g6_list.append(row)

# Liste in ein Pandas DataFrame umwandeln
df_g6 = pd.DataFrame(result_g6_list, columns=['Date','Kanton_Kurzname', 'Energiewirkung_GWh_CH','Massnahmenbereich_lang'])
df_g6

,Date,Kanton_Kurzname,Energiewirkung_GWh_CH,Massnahmenbereich_lang
0,2010,https://ld.admin.ch/country/CHE,101.12,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
1,2010,https://ld.admin.ch/country/CHE,3.17,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
2,2010,https://ld.admin.ch/country/CHE,1.44,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
3,2010,https://ld.admin.ch/country/CHE,162.8,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
4,2010,https://ld.admin.ch/country/CHE,9.24,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
...,...,...,...,...
581,2013,https://ld.admin.ch/country/CHE,252.79,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
582,2013,https://ld.admin.ch/country/CHE,194.42,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
583,2012,https://ld.admin.ch/country/CHE,147.62,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
584,2010,https://ld.admin.ch/country/CHE,9.24,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...


In [219]:
# nicht überall sind die Kurznamen hinterlegt, weshalb diese ersetzt werden
df_g6['Kanton_Kurzname'] = df_g6['Kanton_Kurzname'].replace(kanton_dict_lindas.keys() , kanton_dict_lindas.values() , regex=True)

In [220]:
df_g6

,Date,Kanton_Kurzname,Energiewirkung_GWh_CH,Massnahmenbereich_lang
0,2010,CH,101.12,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
1,2010,CH,3.17,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
2,2010,CH,1.44,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
3,2010,CH,162.8,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
4,2010,CH,9.24,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
...,...,...,...,...
581,2013,CH,252.79,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
582,2013,CH,194.42,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
583,2012,CH,147.62,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
584,2010,CH,9.24,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...


In [221]:
df_g6['Massnahmenbereich'] = df_g6['Massnahmenbereich_lang'].apply(extract_last_part)
df_g6

,Date,Kanton_Kurzname,Energiewirkung_GWh_CH,Massnahmenbereich_lang,Massnahmenbereich
0,2010,CH,101.12,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Haustechnik
1,2010,CH,3.17,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Neubau
2,2010,CH,1.44,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Systemsanierung
3,2010,CH,162.8,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Total Energiewirkung Gebäudeprogramm
4,2010,CH,9.24,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Wärmedämmung
...,...,...,...,...,...
581,2013,CH,252.79,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Waermedaemmung
582,2013,CH,194.42,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Zentrale_Waermeversorgung
583,2012,CH,147.62,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Zentrale_Waermeversorgung
584,2010,CH,9.24,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Waermedaemmung


In [222]:
# Trotz distinct hat es einige Doublikate, diese werden gelöscht
df_g6_no_duplicates = df_g6.drop_duplicates()
df_g6_no_duplicates

,Date,Kanton_Kurzname,Energiewirkung_GWh_CH,Massnahmenbereich_lang,Massnahmenbereich
0,2010,CH,101.12,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Haustechnik
1,2010,CH,3.17,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Neubau
2,2010,CH,1.44,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Systemsanierung
3,2010,CH,162.8,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Total Energiewirkung Gebäudeprogramm
4,2010,CH,9.24,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Wärmedämmung
...,...,...,...,...,...
581,2013,CH,252.79,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Waermedaemmung
582,2013,CH,194.42,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Zentrale_Waermeversorgung
583,2012,CH,147.62,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Zentrale_Waermeversorgung
584,2010,CH,9.24,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Waermedaemmung


In [223]:
df_g6_no_duplicates.to_csv('gebaudeprogramm_Energiewirkung.csv', index=False)

### Abfrage: CO2 Wirkung pro Massnahmenbereich

In [224]:
Gebaeudeprogramm_CO2_QUERY = '''
PREFIX schema: <http://schema.org/>

select distinct  ?Date ?Kanton_Kurzname ?CO2_Wirkung_Tonnen_CH ?Massnahmenbereich

where {
        SERVICE <https://lindas.admin.ch/query>
            {?Massnahmenbereich <http://schema.org/amount> ?CO2_Wirkung_Tonnen_CH .
            ?Massnahmenbereich <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_co2wirkung/Jahr> ?Date .
            ?Massnahmenbereich <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_co2wirkung/region> ?Kanton_Kurzname.
}}
'''

# Graph erstellen und SPARQL-Abfrage durchführen
g7 = Graph()
results_g7 = g7.query(Gebaeudeprogramm_CO2_QUERY)

result_g7_list = []
for row in results_g7:
    result_g7_list.append(row)

# Liste in ein Pandas DataFrame umwandeln
df_g7 = pd.DataFrame(result_g7_list, columns=['Date', 'Kanton_Kurzname', 'CO2_Wirkung_Tonnen_CH', 'Massnahmenbereich_lang'])
df_g7

,Date,Kanton_Kurzname,CO2_Wirkung_Tonnen_CH,Massnahmenbereich_lang
0,2010,https://ld.admin.ch/country/CHE,30259,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
1,2010,https://ld.admin.ch/country/CHE,1112,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
2,2010,https://ld.admin.ch/country/CHE,369,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
3,2010,https://ld.admin.ch/country/CHE,46879,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
4,2010,https://ld.admin.ch/country/CHE,1748,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
...,...,...,...,...
378,2020,https://ld.admin.ch/country/CHE,146169,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
379,2021,https://ld.admin.ch/country/CHE,154805,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
380,2021,https://ld.admin.ch/country/CHE,156177,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
381,2022,https://ld.admin.ch/country/CHE,162564,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...


In [225]:
# nicht überall sind die Kurznamen hinterlegt, weshalb diese ersetzt werden
df_g7['Kanton_Kurzname'] = df_g7['Kanton_Kurzname'].replace(kanton_dict_lindas.keys() , kanton_dict_lindas.values() , regex=True)

In [226]:
df_g7

,Date,Kanton_Kurzname,CO2_Wirkung_Tonnen_CH,Massnahmenbereich_lang
0,2010,CH,30259,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
1,2010,CH,1112,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
2,2010,CH,369,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
3,2010,CH,46879,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
4,2010,CH,1748,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
...,...,...,...,...
378,2020,CH,146169,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
379,2021,CH,154805,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
380,2021,CH,156177,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
381,2022,CH,162564,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...


In [227]:

df_g7['Massnahmenbereich'] = df_g7['Massnahmenbereich_lang'].apply(extract_last_part)
df_g7

,Date,Kanton_Kurzname,CO2_Wirkung_Tonnen_CH,Massnahmenbereich_lang,Massnahmenbereich
0,2010,CH,30259,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Haustechnik
1,2010,CH,1112,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Neubau
2,2010,CH,369,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Systemsanierung
3,2010,CH,46879,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Total CO2-Wirkung Gebäudeprogramm
4,2010,CH,1748,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Wärmedämmung
...,...,...,...,...,...
378,2020,CH,146169,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Zentrale_Waermeversorgung
379,2021,CH,154805,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Waermedaemmung
380,2021,CH,156177,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Zentrale_Waermeversorgung
381,2022,CH,162564,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Waermedaemmung


In [228]:
df_g7_no_duplicates = df_g7.drop_duplicates()
df_g7_no_duplicates

,Date,Kanton_Kurzname,CO2_Wirkung_Tonnen_CH,Massnahmenbereich_lang,Massnahmenbereich
0,2010,CH,30259,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Haustechnik
1,2010,CH,1112,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Neubau
2,2010,CH,369,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Systemsanierung
3,2010,CH,46879,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Total CO2-Wirkung Gebäudeprogramm
4,2010,CH,1748,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Wärmedämmung
...,...,...,...,...,...
378,2020,CH,146169,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Zentrale_Waermeversorgung
379,2021,CH,154805,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Waermedaemmung
380,2021,CH,156177,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Zentrale_Waermeversorgung
381,2022,CH,162564,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Waermedaemmung


In [229]:
df_g7_no_duplicates.to_csv('gebaudeprogramm_CO2Wirkung.csv', index=False)

In [237]:

import pandas as pd

# Lade die CSV-Datei in ein DataFrame
df = pd.read_csv('grundlagen_gemeinde_kantone.csv')  # Ersetze 'deine_datei.csv' durch den tatsächlichen Dateinamen

# Extrahiere die einzigartigen PLZ-Werte und entferne NaN-Werte
plz_series = df['PLZ'].dropna().astype(int).unique()

# Konvertiere die Pandas Series in eine Python-Liste
plz_list = plz_series.tolist()

# Ergebnisse anzeigen
print(plz_list)


[1724, 3928, 1275, 8856, 5054, 8514, 1358, 3933, 5079, 3005, 3011, 3014, 3012, 3003, 3010, 3007, 3015, 3018, 3019, 3001, 3008, 3006, 3000, 3004, 3013, 2206, 2207, 2065, 2046, 2053, 2052, 2037, 2054, 2057, 2058, 2208, 2043, 2056, 2063, 1752, 5304, 1653, 2414, 3421, 8854, 6781, 6780, 6614, 5315, 8880, 5245, 8751, 6464, 5642, 2514, 8627, 4402, 8213, 6822, 4522, 4419, 6516, 6597, 6635, 5413, 4562, 8274, 6264, 8266, 1176, 4118, 1737, 8907, 5013, 1439, 1261, 4310, 1699, 1470, 8933, 3280, 1656, 2882, 1474, 1468, 1891, 5733, 6204, 5746, 3315, 1350, 4433, 6418, 7742, 7746, 7745, 7741, 7710, 8902, 7306, 6340, 9633, 1032, 3053, 9200, 3704, 5643, 1272, 3636, 1874, 5503, 3989, 3985, 3998, 4492, 8197, 1971, 1299, 8165, 1110, 5036, 5243, 1973, 1961, 1966, 1981, 6463, 5732, 5415, 8864, 9535, 9105, 3822, 1126, 2829, 2824, 2828, 4656, 4242, 5417, 1682, 5467, 9436, 5024, 1453, 1714, 6377, 8592, 9464, 1515, 1741, 1294, 5072, 3054, 4802, 6928, 3112, 5723, 4332, 1278, 1322, 1356, 8194, 1921, 8466, 5608, 185